In [1]:
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.tools import BaseTool
from subprocess import Popen,PIPE
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [3]:
model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model = model_name, openai_api_key=os.getenv('OPENAI_API_KEY'))
index_name = 'first-index'


In [4]:
from langchain.vectorstores import Pinecone
vector_store = Pinecone.from_existing_index(index_name,embeddings)

In [5]:
# chat completion llm
llm = ChatOpenAI(
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    model_name='gpt-3.5-turbo-1106',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [6]:
class ExecuteCommand(BaseTool):
    name : str =  "execute command",
    description = "executer les commandes dans la machine virtuelle."

    def _run(self, command: str):
        cmd = command.split()
        if cmd[0]=='sudo':
            cmd.insert(1,'-S')
        #print(cmd)        
        p = Popen(cmd ,stdin = PIPE, stdout = PIPE, universal_newlines = True)
        p.stdin.write('test1!')
        res,_ = p.communicate()
    
        p.terminate()

        return res
        
        #return json.dumps(cmd)       

    def _arun(self, webpage: str):
        raise NotImplementedError("Cet outil ne support pas async")

execute_command = ExecuteCommand()

In [7]:
class Transfrom_to_Bash(BaseTool):
    name : str = "Transform to Bash",
    description= "Transformer les textes en commandes bash qui vont être executées dans la machine virtuelle."

    def _run(self, command: str):
        return command

    def _arun(self, webpage: str):
        raise NotImplementedError("Cet outil ne support pas async")

TransfromtoBash = Transfrom_to_Bash()

In [8]:
from langchain.agents import Tool

tools = [
    Tool(
        name = 'Knowledge Base',
        func=qa.invoke,
        description=(
            """Utilise cet outil pour répondre aux qestions."""
        )
    ),
    Tool(
        name = 'Execute Command',
        func=execute_command._run,
        description=(
            "executer les commandes dans la machine virtuelle."
        )
    ),
    Tool(
        name = 'Transform to Bash',
        func=TransfromtoBash._run,
        description=(
            "Transformer les textes en commandes bash qui vont être executées dans la machine virtuelle."
        )
    )
    
]

In [9]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=conversational_memory
)

c:\Users\zikri\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
fixed_prompt = """ 
                Vous êtes un agent IA qui doit répondre à des questions. Commencez par utiliser les outils disponibles pour répondre à la question. Vous devez répondre à la question avec un plan détaillé en commandes bash. 
                Souvenez-vous de répondre à la question en retournant des commandes bash selon la question.
                """
agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [11]:
inp = []
memory =[]
out = []
while True:
    query = input("Ask me anything: ")
    if query =='':
        break
    rest = agent(query)
    inp.append(rest['input'])
    memory.append(rest['chat_history'])
    out.append(rest['output'])    

c:\Users\zikri\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "How to protect your business"
}
```
Observation: {'query': 'How to protect your business', 'result': "To protect your business, you should have a plan in place for staying up and running after a ransomware attack. This plan should be put in writing and shared with everyone who needs to know. Regularly back up your data and ensure that those backups are not connected to the network. Keep your security up to date by installing the latest patches and updates, and look for additional means of protection like email authentication and intrusion prevention software. It's important to alert your staff and teach them how to avoid phishing scams and spot common ways computers and devices become infected. Additionally, deploying a safety net, such as email authentication technology, can help prevent phishing emails from reaching your employees. Creating a culture of security by implementing regu

In [12]:
out

["To protect your business, you should have a plan in place for staying up and running after a ransomware attack. This plan should be put in writing and shared with everyone who needs to know. Regularly back up your data and ensure that those backups are not connected to the network. Keep your security up to date by installing the latest patches and updates, and look for additional means of protection like email authentication and intrusion prevention software. It's important to alert your staff and teach them how to avoid phishing scams and spot common ways computers and devices become infected. Additionally, deploying a safety net, such as email authentication technology, can help prevent phishing emails from reaching your employees. Creating a culture of security by implementing regular employee training and having a plan for saving data, running the business, and notifying customers in case of a breach are also important steps to protect your business. For more information, you can